<a href="https://colab.research.google.com/github/j28302830/Senior_Project/blob/main/reconized_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Samples and Models file

In [143]:
!wget http://dataset.tlm.unavarra.es/ransomware/samplesAndModels.tar.xz
!tar -Jxvf /content/samplesAndModels.tar.xz &> /dev/null

--2022-05-16 14:06:32--  http://dataset.tlm.unavarra.es/ransomware/samplesAndModels.tar.xz
Resolving dataset.tlm.unavarra.es (dataset.tlm.unavarra.es)... 130.206.160.87
Connecting to dataset.tlm.unavarra.es (dataset.tlm.unavarra.es)|130.206.160.87|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41519408 (40M) [application/x-xz]
Saving to: ‘samplesAndModels.tar.xz.3’

samplesAndModels.ta 100%[===================>]  39.60M  7.47MB/s    in 5.9s    

2022-05-16 14:06:38 (6.71 MB/s) - ‘samplesAndModels.tar.xz.3’ saved [41519408/41519408]



# Import Package

In [173]:
import tensorflow as tf
import keras
import glob
import numpy as np
from pickle import load
import joblib 

from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import plotly.express as px
pd.set_option('max_rows', 99999)

# Predefined Function

In [145]:
def load_sample(local_path):
  filenames = glob.glob(local_path + "/*.txt")
  for i,filename in enumerate(filenames):
    if i == 0:
      x = np.loadtxt(filename, delimiter=',', dtype = 'float32')
    else: 
      ds = np.loadtxt(filename, delimiter=',', dtype = 'float32')
      x = np.concatenate((x, ds), axis=0)

  return x

In [146]:
def metrics(model , x, y):
  predict = model.predict(x)
  predict[predict >= 0.99] = 1
  predict[predict < 0.99] = 0

  true_values = y
  predictions = predict

  TP = ((predictions == 1) & (true_values == 1)).sum()
  FP = ((predictions == 1) & (true_values == 0)).sum()
  TN = ((predictions == 0) & (true_values == 0)).sum()
  FN = ((predictions == 0) & (true_values == 1)).sum()

  accuracy = (TP+TN) / (TP+FP+TN+FN)
  precision = TP / (TP+FP)
  recall = TP / (TP+FN)
  f_measure = 2*(precision*recall/(precision+recall))
  temp = ((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))** 0.5
  phi_coefficient = (TP*TN-FP*FN) / temp
  
  #mat = confusion_matrix(true_values,predictions)

  #print("accuracy score:", accuracy_score(true_values, predictions))
    
  #sns.heatmap(mat,square= True, annot=True, cbar= False)
  #plt.xlabel("predicted value")
  #plt.ylabel("true value")
  #plt.show()
  
  evaluate = model.evaluate(x=x, y=y)

  return [evaluate[1], accuracy, precision, recall, f_measure, phi_coefficient]

In [147]:
def scale(feature):
  scaler = joblib.load('/content/scaler.scaler')
  feature_scaled = scaler.transform(feature)
  feature_scaled_10X3 = np.reshape(feature_scaled, (feature_scaled.shape[0], 10, 3))
  return feature_scaled, feature_scaled_10X3

# Load NN, CNN, and LSTM model

In [148]:
model_NN = tf.keras.models.load_model('/content/NN_CNN_LSTM_Comparison/NN')
model_CNN = tf.keras.models.load_model('/content/NN_CNN_LSTM_Comparison/CNN')
model_LSTM = tf.keras.models.load_model('/content/NN_CNN_LSTM_Comparison/LSTM')

JSON

In [149]:
json_file = open('/content/NN.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)

# Load sample

In [150]:
index = list(range(0, 30)) #show range of feature index 
test = load_sample(local_path = r'/content/samples/ransomwareSamples_test')
train = load_sample(local_path = r'/content/samples/ransomwareSamples_train')
all_ransomware = np.concatenate( (train, test), axis = 0 )
day_zero = np.loadtxt('/content/samples/userSamples/day0.txt', delimiter=',', dtype = 'float32')

# Preprocessing Sample and Run Model 

### day0 and all ransomware sample

In [201]:
label = np.delete(all_ransomware, index + [31], axis=1)
feature = np.delete(all_ransomware, [30,31], axis=1)


label2 = np.delete(day_zero, index, axis=1)
feature2 = np.delete(day_zero, [30], axis=1)

label = np.concatenate( (label, label2), axis = 0 )
feature = np.concatenate( (feature, feature2), axis = 0 )

feature_scaled, feature_scaled_10X3 = scale(feature)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning:

Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations



In [202]:
NN = metrics(model_NN, x=feature_scaled, y=label)
CNN = metrics(model_CNN, x=feature_scaled_10X3, y=label)
LSTM = metrics(model_LSTM, x=feature_scaled_10X3, y=label)
print("NN:\naccuracy(): %f" %NN[1] )
print("CNN:\naccuracy(): %f" %CNN[1] )
print("LSTM:\naccuracy(): %f" %LSTM[1] )

   26/31611 [..............................] - ETA: 1:04 - loss: 0.0128 - accuracy: 0.9988     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning:

overflow encountered in long_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning:

invalid value encountered in power



31611/31611 [==============================] - 126s 4ms/step - loss: 0.4148 - accuracy: 0.9658
NN:
accuracy(): 0.966149
CNN:
accuracy(): 0.965596
LSTM:
accuracy(): 0.965573


### test sample without low activity data

In [193]:
label = np.delete(test, index + [31], axis=1)
feature = np.delete(test, [30,31], axis=1)

count = np.count_nonzero(feature == 0, axis=1)
low_activity = np.where( count >= (feature.shape[1]/3) )

label = np.delete(label, low_activity, axis=0)
feature = np.delete(feature, low_activity, axis=0)

feature_scaled, feature_scaled_10X3 = scale(feature)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning:

Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations



In [194]:
NN = metrics(model_NN, x=feature_scaled, y=label)
CNN = metrics(model_CNN, x=feature_scaled_10X3, y=label)
LSTM = metrics(model_LSTM, x=feature_scaled_10X3, y=label)
print("NN:\naccuracy(): %f" %NN[1] )
print("CNN:\naccuracy(): %f" %CNN[1] )
print("LSTM:\naccuracy(): %f" %LSTM[1] )

 46/292 [===>..........................] - ETA: 0s - loss: 0.0376 - accuracy: 0.9959

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in true_divide



292/292 [==============================] - 1s 4ms/step - loss: 0.1293 - accuracy: 0.9803
NN:
accuracy(): 0.952519
CNN:
accuracy(): 0.937192
LSTM:
accuracy(): 0.946838


### train sample

In [195]:
label = np.delete(train, index + [31], axis=1)
feature = np.delete(train, [30,31], axis=1)

feature_scaled, feature_scaled_10X3 = scale(feature)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning:

Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations



In [196]:
NN = metrics(model_NN, x=feature_scaled, y=label)
CNN = metrics(model_CNN, x=feature_scaled_10X3, y=label)
LSTM = metrics(model_LSTM, x=feature_scaled_10X3, y=label)
print("NN:\naccuracy(): %f" %NN[1] )
print("CNN:\naccuracy(): %f" %CNN[1] )
print("LSTM:\naccuracy(): %f" %LSTM[1] )

 49/361 [===>..........................] - ETA: 0s - loss: 0.0068 - accuracy: 0.9994

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in true_divide



361/361 [==============================] - 2s 5ms/step - loss: 0.0799 - accuracy: 0.9894
NN:
accuracy(): 0.986030
CNN:
accuracy(): 0.984035
LSTM:
accuracy(): 0.974230


### all ransomware sample

In [197]:
index = list(range(0, 30))

label = np.delete(train, index + [31], axis=1)
feature = np.delete(train, [30,31], axis=1)

label2 = np.delete(test, index + [31], axis=1)
feature2 = np.delete(test, [30,31], axis=1)

label = np.concatenate( (label, label2), axis = 0 )
feature = np.concatenate( (feature, feature2), axis = 0 )

count = np.count_nonzero(feature == 0, axis=1)
low_activity = np.where( count >= (feature.shape[1]/3) )

label = np.delete(label, low_activity, axis=0)
feature = np.delete(feature, low_activity, axis=0)

feature_scaled, feature_scaled_10X3 = scale(feature)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning:

Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations



In [198]:
NN = metrics(model_NN, x=feature_scaled, y=label)
CNN = metrics(model_CNN, x=feature_scaled_10X3, y=label)
LSTM = metrics(model_LSTM, x=feature_scaled_10X3, y=label)
print("NN:\naccuracy(): %f" %NN[1] )
print("CNN:\naccuracy(): %f" %CNN[1] )
print("LSTM:\naccuracy(): %f" %LSTM[1] )

 52/652 [=>............................] - ETA: 1s - loss: 0.0064 - accuracy: 0.9994

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in true_divide



652/652 [==============================] - 2s 4ms/step - loss: 0.1020 - accuracy: 0.9853
NN:
accuracy(): 0.971038
CNN:
accuracy(): 0.963078
LSTM:
accuracy(): 0.961976


### test sample(each ransomware) without low activity data 

In [178]:
category = []
sample = []
local_path = r'/content/samples/ransomwareSamples_test'
filenames = glob.glob(local_path + "/*.txt")
for i,filename in enumerate(filenames):
  name = filename.replace(local_path + "/",'').replace('.txt', '')
  sample.append(name[0:name.find('_')])
  

In [179]:
category = sorted(set(sample))
print(category)

['Aleta', 'CTBLocker', 'CryLock', 'CrypMIC', 'CryptFile2', 'CryptoFortress', 'CryptoMix', 'CryptoShield', 'Crysis', 'Cryxox', 'DMALocker', 'Dharma', 'Diablo6', 'Eris', 'GandCrab', 'GlobeImposter', 'Jaff', 'Locky', 'MRCR', 'Maze', 'Mole', 'Odin', 'Phobos', 'RansomX', 'Razi', 'Ryuk', 'Sage', 'Scarab', 'Shade', 'Shaofao', 'Sodinokibi', 'Spora', 'Stop', 'TeslaCrypt', 'Virlock', 'Zeus', 'bart', 'bitPaymer', 'cerber', 'maktub', 'netwalker', 'revenge', 'wannaCry']


In [203]:
filenames = glob.glob(local_path + "/*.txt")
for i, name in enumerate(category):
  indices = [i for i, x in enumerate(sample) if x == name]
  for j, pos in enumerate(indices):
    if j == 0:
      x = np.loadtxt(filenames[pos], delimiter=',', dtype = 'float32')
    else: 
      ds = np.loadtxt(filenames[pos], delimiter=',', dtype = 'float32')
      x = np.concatenate((x, ds), axis=0)
  
  label = np.delete(x, index + [31], axis=1)
  feature = np.delete(x, [30,31], axis=1)
  count = np.count_nonzero(feature == 0, axis=1)
  low_activity = np.where( count >= (feature.shape[1]/3) )
  label = np.delete(label, low_activity, axis=0)
  feature = np.delete(feature, low_activity, axis=0)
  feature_scaled, feature_scaled_10X3 = scale(feature)

  NN = metrics(model_NN, x=feature_scaled, y=label)
  CNN = metrics(model_CNN, x=feature_scaled_10X3, y=label)
  LSTM = metrics(model_LSTM, x=feature_scaled_10X3, y=label)

  if i == 0:
    NN_data = [[name, NN[1], NN[2], NN[3], NN[4]]]
    CNN_data = [[name, CNN[1], CNN[2], CNN[3], CNN[4]]]
    LSTM_data = [[name, LSTM[1], LSTM[2], LSTM[3], LSTM[4]]]
  else:
    NN_data = NN_data + [[name, NN[1], NN[2], NN[3], NN[4]]]
    CNN_data = CNN_data + [[name, CNN[1], CNN[2], CNN[3], CNN[4]]]
    LSTM_data = LSTM_data + [[name, LSTM[1], LSTM[2], LSTM[3], LSTM[4]]]
  

6/6 [==============================] - 0s 3ms/step - loss: 1.7671 - accuracy: 0.6380


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning:

Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in true_divide



3/3 [==============================] - 0s 5ms/step - loss: 2.6289 - accuracy: 0.7447


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning:

Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in true_divide



3/3 [==============================] - 0s 5ms/step - loss: 0.6515 - accuracy: 0.9500


...

3/3 [==============================] - 0s 5ms/step - loss: 0.0017 - accuracy: 1.0000


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning:

Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in true_divide



 1/15 [=>............................] - ETA: 0s - loss: 0.0339 - accuracy: 0.9688

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning:

Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in true_divide



15/15 [==============================] - 0s 4ms/step - loss: 0.0251 - accuracy: 0.9978


In [204]:
NN_df = pd.DataFrame(NN_data, columns = ['Name','Accuracy','Precision', 'Recall', 'F_measure'])

px.bar(NN_df, x='Name', y=['Accuracy'], barmode='group', title='NN')

In [205]:
CNN_df = pd.DataFrame(CNN_data, columns = ['Name','Accuracy','Precision', 'Recall', 'F_measure'])

px.bar(CNN_df, x='Name', y=['Accuracy'], barmode='group', title='CNN')

In [206]:
LSTM_df = pd.DataFrame(LSTM_data, columns = ['Name','Accuracy','Precision', 'Recall', 'F_measure'])

px.bar(LSTM_df, x='Name', y=['Accuracy'], barmode='group', title='LSTM')